Chuyển từ **Ridge Regression** sang **Time-Series LSTM** yêu cầu một số thay đổi trong việc xử lý dữ liệu và xây dựng mô hình. Dưới đây là hướng dẫn và một phiên bản cập nhật cho class để sử dụng **LSTM**:

---

### **Các bước chuyển đổi:**

1. **Tiền xử lý dữ liệu cho LSTM:**
   - LSTM yêu cầu dữ liệu đầu vào có định dạng **3D**: `(samples, timesteps, features)`.
   - Chuyển dữ liệu `X_train` và `X_test` thành các **sequences** (cửa sổ thời gian).

2. **Xây dựng mô hình LSTM:**
   - Sử dụng Keras hoặc TensorFlow để xây dựng một mạng LSTM.
   - Kích thước đầu vào (input shape) phải phù hợp với số lượng timesteps và features.

3. **Đào tạo mô hình LSTM:**
   - Chia dữ liệu thành `train` và `test` với `shuffle=False`.
   - Sử dụng callback (ví dụ: EarlyStopping) để tránh overfitting.

4. **Đánh giá và trực quan hóa kết quả:**
   - Tính các chỉ số như `R2`, `MSE`, và `MAPE`.
   - Vẽ các biểu đồ tương tự như trước.

---

### **Cập nhật class để sử dụng LSTM:**

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from matplotlib.ticker import FuncFormatter

class TimeSeriesLSTMModel:
    def __init__(self, file_path, look_back=10):
        """
        Initialize the class with the file path of the dataset and look-back window.
        """
        self.file_path = file_path
        self.look_back = look_back
        self.data = None
        self.model = None
        self.scaler = MinMaxScaler(feature_range=(0, 1))

    def load_and_preprocess_data(self):
        """
        Load the dataset and preprocess the data for LSTM.
        """
        # Load data
        self.data = pd.read_csv(self.file_path)
        self.data["close_tomor"] = self.data["close"].shift(-1)
        self.data = self.data.iloc[:-1]
        
        # Scaling data
        self.data_scaled = self.scaler.fit_transform(self.data[['close_tomor']])
        
        # Create sequences
        X, y = [], []
        for i in range(self.look_back, len(self.data_scaled)):
            X.append(self.data_scaled[i - self.look_back:i, 0])  # Sequence of look_back days
            y.append(self.data_scaled[i, 0])  # Target value

        X, y = np.array(X), np.array(y)
        X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape to (samples, timesteps, features)
        
        # Split into training and testing sets
        train_size = int(len(X) * 0.75)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        return X_train, X_test, y_train, y_test

    def build_model(self):
        """
        Build the LSTM model.
        """
        self.model = Sequential([
            LSTM(50, activation='relu', return_sequences=True, input_shape=(self.look_back, 1)),
            LSTM(50, activation='relu'),
            Dense(1)
        ])
        self.model.compile(optimizer='adam', loss='mse')

    def train_model(self, X_train, y_train, epochs=50, batch_size=32):
        """
        Train the LSTM model.
        """
        early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stop])

    def evaluate_model(self, y_true, y_pred):
        """
        Evaluate the model performance.
        """
        y_true = self.scaler.inverse_transform(y_true.reshape(-1, 1))
        y_pred = self.scaler.inverse_transform(y_pred.reshape(-1, 1))

        print("R2:", r2_score(y_true, y_pred))
        print("MSE:", mean_squared_error(y_true, y_pred))
        print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))
        print("MAE:", mean_absolute_error(y_true, y_pred))
        print("MAPE%:", f"{mean_absolute_percentage_error(y_true, y_pred) * 100:.2f}%")

    def plot_predictions(self, y_test, y_pred):
        """
        Plot predictions vs actual values.
        """
        y_test = self.scaler.inverse_transform(y_test.reshape(-1, 1))
        y_pred = self.scaler.inverse_transform(y_pred.reshape(-1, 1))

        plt.figure(figsize=(10, 5))
        plt.plot(y_test, label="Actual")
        plt.plot(y_pred, label="Predicted", linestyle="--")
        plt.title("Test Predictions")
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.legend()
        plt.show()

    def process(self):
        """
        Execute the full pipeline: load, preprocess, train, evaluate, and plot.
        """
        X_train, X_test, y_train, y_test = self.load_and_preprocess_data()
        self.build_model()
        self.train_model(X_train, y_train)
        y_pred_train = self.model.predict(X_train)
        y_pred_test = self.model.predict(X_test)
        self.evaluate_model(y_test, y_pred_test)
        self.plot_predictions(y_test, y_pred_test)

# Example usage for one file
file_path = "D:/OneDrive - Hanoi University of Science and Technology/GIT/MiniProj_StockPrediction_ML_SpManhGraduationProj_2024/data/raw20192024/FPT_stock_data.csv"
model = TimeSeriesLSTMModel(file_path)
model.process()
```

---

### **Mô tả chính trong Class:**
1. **`load_and_preprocess_data`:**
   - Tạo các cửa sổ thời gian (`look_back`) cho LSTM.
   - Chuẩn hóa dữ liệu bằng `MinMaxScaler`.

2. **`build_model`:**
   - Xây dựng mạng LSTM với 2 lớp LSTM và 1 lớp Dense.

3. **`train_model`:**
   - Đào tạo mô hình với `EarlyStopping` để tránh overfitting.

4. **`evaluate_model`:**
   - Tính các chỉ số đánh giá và in kết quả.

5. **`plot_predictions`:**
   - Vẽ biểu đồ so sánh giữa giá trị thực tế và giá trị dự đoán.

---

### **Lưu ý:**
- Đảm bảo rằng file CSV có cột `"close"` để tạo các sequences thời gian.
- Kiểm tra `look_back` phù hợp (ví dụ: 10 ngày gần nhất).

Bạn có thể chạy và kiểm tra kết quả trực tiếp từ LSTM model này! 😊

### **Cập nhật để thêm `R2` và `DA` (Directional Accuracy)**
Để thêm các chỉ số **R2** và **DA (Directional Accuracy)**, chúng ta chỉ cần mở rộng hàm đánh giá và trực quan hóa mô hình.

### **Cập nhật Class để cải thiện:**

#### **1. Tính `R2` và `DA`:**
- **R2:** Đã có trong thư viện `sklearn.metrics`.
- **DA:** Tự tính bằng cách so sánh hướng biến động giữa giá trị thực (`y_true`) và giá trị dự đoán (`y_pred`).

#### **2. Hợp nhất biểu đồ:**
- Thay vì tách ra hai biểu đồ (scatter và line), ta có thể hiển thị chúng trên một cửa sổ với hai subplot.

#### **3. Cập nhật Code:**
```python
class TimeSeriesLSTMModel:
    def __init__(self, file_path, look_back=10):
        """
        Initialize the class with the file path of the dataset and look-back window.
        """
        self.file_path = file_path
        self.look_back = look_back
        self.data = None
        self.model = None
        self.scaler = MinMaxScaler(feature_range=(0, 1))

    def load_and_preprocess_data(self):
        """
        Load the dataset and preprocess the data for LSTM.
        """
        # Load data
        self.data = pd.read_csv(self.file_path)
        self.data["close_tomor"] = self.data["close"].shift(-1)
        self.data = self.data.iloc[:-1]
        
        # Scaling data
        self.data_scaled = self.scaler.fit_transform(self.data[['close_tomor']])
        
        # Create sequences
        X, y = [], []
        for i in range(self.look_back, len(self.data_scaled)):
            X.append(self.data_scaled[i - self.look_back:i, 0])  # Sequence of look_back days
            y.append(self.data_scaled[i, 0])  # Target value

        X, y = np.array(X), np.array(y)
        X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape to (samples, timesteps, features)
        
        # Split into training and testing sets
        train_size = int(len(X) * 0.75)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]

        return X_train, X_test, y_train, y_test

    def build_model(self):
        """
        Build the LSTM model.
        """
        self.model = Sequential([
            LSTM(50, activation='relu', return_sequences=True, input_shape=(self.look_back, 1)),
            LSTM(50, activation='relu'),
            Dense(1)
        ])
        self.model.compile(optimizer='adam', loss='mse')

    def train_model(self, X_train, y_train, epochs=50, batch_size=32):
        """
        Train the LSTM model.
        """
        early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stop])

    def evaluate_model(self, y_true, y_pred):
        """
        Evaluate the model performance.
        """
        y_true_rescaled = self.scaler.inverse_transform(y_true.reshape(-1, 1))
        y_pred_rescaled = self.scaler.inverse_transform(y_pred.reshape(-1, 1))

        # Calculate R2
        r2 = r2_score(y_true_rescaled, y_pred_rescaled)

        # Calculate Directional Accuracy
        da = np.mean(
            np.sign(y_true_rescaled[1:] - y_true_rescaled[:-1]) ==
            np.sign(y_pred_rescaled[1:] - y_pred_rescaled[:-1])
        )

        print("R2:", r2)
        print("Directional Accuracy (DA):", f"{da * 100:.2f}%")
        print("MSE:", mean_squared_error(y_true_rescaled, y_pred_rescaled))
        print("RMSE:", np.sqrt(mean_squared_error(y_true_rescaled, y_pred_rescaled)))
        print("MAE:", mean_absolute_error(y_true_rescaled, y_pred_rescaled))
        print("MAPE%:", f"{mean_absolute_percentage_error(y_true_rescaled, y_pred_rescaled) * 100:.2f}%")

        return r2, da

    def plot_predictions(self, y_test, y_pred, num_samples=50):
        """
        Plot predictions vs actual values with both scatter and line plots.
        """
        y_test_rescaled = self.scaler.inverse_transform(y_test.reshape(-1, 1))
        y_pred_rescaled = self.scaler.inverse_transform(y_pred.reshape(-1, 1))

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

        # Scatter plot
        ax1.scatter(y_test_rescaled, y_pred_rescaled, alpha=0.5)
        ax1.set_title("Predictions vs Actual")
        ax1.set_xlabel("Actual Values")
        ax1.set_ylabel("Predicted Values")

        # Add y=x line
        min_val = min(y_test_rescaled.min(), y_pred_rescaled.min())
        max_val = max(y_test_rescaled.max(), y_pred_rescaled.max())
        ax1.plot([min_val, max_val], [min_val, max_val], 'r--')

        # Line plot
        x_points = range(num_samples)
        ax2.plot(x_points, y_test_rescaled[:num_samples], 'b-', label='Actual')
        ax2.plot(x_points, y_pred_rescaled[:num_samples], 'orange', linestyle='--', label='Predicted')
        
        ax2.set_title("Test Predictions")
        ax2.set_xlabel("Point")
        ax2.set_ylabel("Value")
        ax2.legend()

        plt.tight_layout()
        plt.show()

    def process(self):
        """
        Execute the full pipeline: load, preprocess, train, evaluate, and plot.
        """
        X_train, X_test, y_train, y_test = self.load_and_preprocess_data()
        self.build_model()
        self.train_model(X_train, y_train)
        y_pred_train = self.model.predict(X_train)
        y_pred_test = self.model.predict(X_test)
        self.evaluate_model(y_test, y_pred_test)
        self.plot_predictions(y_test, y_pred_test)

# Example usage for one file
file_path = "D:/OneDrive - Hanoi University of Science and Technology/GIT/MiniProj_StockPrediction_ML_SpManhGraduationProj_2024/data/raw20192024/FPT_stock_data.csv"
model = TimeSeriesLSTMModel(file_path)
model.process()
```

---

### **Các thay đổi chính:**
1. **Thêm `R2` và `Directional Accuracy`:**
   - **R2**: Đánh giá độ phù hợp của mô hình.
   - **DA**: Đo lường khả năng dự đoán đúng hướng biến động giá trị.

2. **Biểu đồ hợp nhất:**
   - Scatter plot (trục `y_true` và `y_pred`) và line plot (biểu diễn mẫu) trong cùng một cửa sổ.

3. **Gọn gàng hơn:**
   - Dùng `min()` và `max()` cho đường y=x trong scatter plot.
   - Hiển thị số lượng mẫu tùy chọn trên line plot (`num_samples`).

---

### **Kết quả mong đợi:**
- **Biểu đồ scatter:** So sánh giá trị dự đoán với thực tế, kèm đường y=x để kiểm tra tính chính xác.
- **Biểu đồ line plot:** Hiển thị xu hướng của mẫu thực và dự đoán, dễ kiểm tra sự sai lệch.
- **Chỉ số R2 và DA:** Được in rõ ràng trong quá trình đánh giá.

Hãy thử áp dụng và kiểm tra kết quả nhé! 😊